## 补充
* https://www.cnblogs.com/go-ahead-wsg/p/12497806.html

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据集
mnist = input_data.read_data_sets("../MNIST_data", one_hot=True) 


Extracting ../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
#输入图片是28*28
n_inputs = 28 # 每次输入一行的长度 取决于图片一行28像素
max_time = 28 # 一共28行(图片看成一个序列)
lstm_size = 100 # 100个隐层单元
n_classes = 10 # 10个分类  
batch_size = 50 # 每批次50样本
n_batch = mnist.train.num_examples // batch_size#计算一共多少批次 

#这里的none表示第一个维度可以任意，取决于放多少张图片
x = tf.placeholder(tf.float32,[None,784])
#正确的标签
y = tf.placeholder(tf.float32,[None,10]) # None 每批次50样本

#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1)) # [100隐层， 10分类]
#初始化偏置值
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义RNN网络
def RNN(X,Weights,Biases):
    #inputs = [batch_size,max_time,n_inputs] 格式
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size) # lstm_size 100个隐层单元
    #final_state[0]是cell state   [0]每个cell记忆的值
    #final_state[1]是hidden_state   [1]最后一次输出的结果
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32) # 格式
    results = tf.nn.softmax(tf.matmul(final_state[1],Weights)+Biases)
    return results
#计算RNN的返回结果
prediction = RNN(x,weights,biases)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#把结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch) + ",Testing Accuracy=" + str(acc))

W0802 16:15:50.835549 12292 deprecation.py:323] From <ipython-input-3-03388ad4c8f0>:24: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0802 16:15:50.838550 12292 deprecation.py:323] From <ipython-input-3-03388ad4c8f0>:27: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0802 16:15:50.998559 12292 deprecation.py:506] From C:\Users\hasee\AppData\Local\Programs\Python\Python35\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument